Create Dateset

In [1]:
from janome.tokenizer import Tokenizer

# データ読み込み
data_path = "D:\Research\ExplainDetecter\dataset.txt"
file = open(data_path , "r" , encoding= "UTF-8")
data = file.read()
file.close()

# 改行でデータを分割
sentences = data.splitlines()

# トークナイザを作る
tokenizer = Tokenizer()

# 単語ごとに分かれたデータを持っている
word_list =[] # List[List[str]]

# 単語ごとに分割し、そのリストをword_listに追加
for sentence in sentences:
    tokens = tokenizer.tokenize(sentence)
    tmp_words = []
    for token in tokens:
        # print(token.part_of_speech)
        tmp_words.append(token.base_form)
        
    word_list.append(tmp_words)

# print(word_list[:])


Word2Vec

In [2]:
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
import numpy as np
w2v_size = 64

# iter:機械学習の繰り返し回数（デフォルトは５）十分に学習できていないときに調整

# Word2Vecの学習　ここで
w2v = Word2Vec(word_list,vector_size = w2v_size,min_count = 5, window = 5,epochs = 5)

#  words = list(w2v.wv.index_to_key)


# word_index = {key:int(key) for key in w2v.wv.key_to_index}
# word_index['0'] = 0
# emb_mat = get_embedding_matrix(w2v , word_index)
# emb.mat.shape

tokenizer_keras = Tokenizer()
tokenizer_keras.fit_on_texts(word_list)
word_index = tokenizer_keras.word_index
num_words = len(word_index)
embedding_matrix = np.zeros((num_words+1, 64))
for word, i in word_index.items():
    if word in w2v.wv.index_to_key:
        embedding_matrix[i] = w2v.wv[word]



C:\Users\e1842\anaconda3\envs\tensorflow-gpu\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:

from keras.layers import Embedding

def get_embedding_matrix(model , word_index):
    embedding_matrix = np.zeros((max(list(word_index.values())) + 1, model.vector_size),dtype = "float32")
    
    for word , labale in word_index.items():
        try:
            embedding_matrix[label] = model.wv[word]
        except KeyError:
            pass
    return embedding_matrix
    

In [4]:
num_words,w2v_size = embedding_matrix.shape

Embedding(num_words , w2v_size,
         weights=[embedding_matrix],
         input_length=128,
         trainable=False)



LSTM

In [5]:

from keras.models import Sequential
import pandas as pd
from tensorflow.keras.layers import LSTM
from keras.layers import Activation, Dense

# max_len = pd.DataFrame(word_list)[0].apply(lambda x:len(x)).max()
max_len = 110
num_words, w2v_size = embedding_matrix.shape

model = Sequential()
# 110は最大入力単語数　つまりごり押し
model.add(Embedding(num_words,w2v_size,
                   weights = [embedding_matrix],
                   input_length=max_len,
                   trainable=False))
model.add(LSTM(128,batch_input_shape=(None,max_len,w2v_size),return_sequences=False))
model.add(Dense(8))
model.add(Activation("softmax"))

model.summary()

#  データセット読み込んで学習してみよう！

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 110, 64)           25984     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
Total params: 125,832
Trainable params: 99,848
Non-trainable params: 25,984
_________________________________________________________________
